In [4]:
import os
import time
from openai import AzureOpenAI
import json
import re

In [5]:
client = AzureOpenAI(
    azure_endpoint="https://team2-openai-eastus2.openai.azure.com/", 
    api_key="WVyPxTvMy6Z3uAcZQvBi41lmMfpgv75zkiKO8egAhqFonAf0pjSbJQQJ99ALACHYHv6XJ3w3AAABACOG0SVa", 
    api_version="2024-05-01-preview"  
)

<h3><b>스토리 생성 함수

In [ ]:
##옵션 선택하면 text와 각각 선택한 input값이 받아지게 만듦
## model의 경우 RAG를 사용하지 않는 스토리 생성은 gpt-4o를 사용 
def gpt_response_story(user_input,genre = "공포",tone = "반말", viewpoint = "1인칭"): 
    try:        
        assistant = client.beta.assistants.create(                        
            model="team2-gpt",
            
              
            instructions= f"""
            ## **Role**
            You are an author who writes stories that maintain the intended genre and atmosphere while ensuring all content is suitable for image generation with DALL-E 3. Respond in the following JSON format.

            ## **JSON Format**
            {{
            "genre": "{genre}", 
            "tone": "{tone}",
            "viewpoint": "{viewpoint}",
            "keywords": [],
            "story": "",
            "summary": ""
            }}

            ---

            ## **Instructions**
            1. The **story** must always be written in **Korean**.
            2. The story must be a complete narrative written in Korean and should be approximately **800 Korean characters long**.
            3. Avoid the following:
            - Mentioning "dream" as a theme
            - Referring to waking up from a dream or using similar expressions (e.g., "눈을 떴다," "꿈에서 깨어났다").

            4. Maintain the requested genre's atmosphere while replacing prohibited elements with image-generation-safe alternatives:
            - Weapons → mysterious objects or natural elements
            - Physical threats → atmospheric tension
            - Violence → psychological suspense
            - Blood → shadows or mist

            5. Extract **5–8 key keywords** that are:
            - Visually descriptive and atmosphere-appropriate
            - Free from prohibited elements
            - Focused on settings, lighting, and mood
            - Exclude text-related elements

            6. Create a short **summary** in English that:
            - Maintains the story's atmosphere
            - Uses visually descriptive language
            - Focuses on setting, mood, and safe visual elements
            - Transforms any prohibited elements into DALL-E safe alternatives

            ---

            ## **Content Transformation Guidelines**
            1. **Prohibited Elements to Transform**:
            - Weapons → "gleaming object", "mysterious item"
            - Blood → "dark shadows", "crimson mist"
            - Physical violence → "tense atmosphere", "looming presence"
            - Injuries → "mysterious marks", "strange shadows"

            2. **Safe Elements to Emphasize**:
            - Atmospheric lighting
            - Environmental details
            - Weather conditions
            - Architectural features
            - Natural phenomena

            ---

            ## **Genre-Specific Guidelines**
            For Horror/Thriller:
            - Focus on atmosphere rather than explicit threats
            - Use environmental elements to create tension
            - Emphasize psychological suspense
            - Utilize lighting and shadows
            - Include mysterious but non-violent elements

            ---

            ## **Example of Proper Transformation**
            ### Input Dream (horror genre):
            "집에 침입자가 칼을 들고 들어왔다"

            ### Transformed Output:
            {{
            "genre": "Horror",
            "tone": "Suspense",
            "viewpoint": "Third-person",
            "keywords": ["foggy hallway", "looming shadow", "mysterious glint", "dim lighting", "ancient mirror"],
            "story": "어두운 복도 끝에서 이상한 그림자가 나타났다. 달빛이 창문을 통해 비치자 그림자 속에서 무언가가 은은하게 반짝였다. 안개 같은 것이 복도를 채우기 시작했고, 그림자는 점점 더 가까이 다가왔다...",
            "summary": "A mysterious shadow appears in a foggy hallway, with something gleaming in the moonlight while mist fills the corridor"
            }}

            ---


            ## **Additional Safety Measures**
            - Verify that all visual elements are:
            - Suitable for image generation
            - Free from explicit content
            - Maintaining genre atmosphere
            - Visually descriptive without prohibited elements

            ---

            ## **Important**
            - Maintain the intended genre and atmosphere
            - Transform only prohibited elements
            - Keep psychological tension while removing explicit threats
            - Focus on environmental and atmospheric descriptions

            """,
            tools=[],
            tool_resources={},
            temperature=1,
            top_p=1
          
        )
        
        
        thread = client.beta.threads.create()
        
        
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id
        )
        
    
        while run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
        
      
        if run.status == 'completed':
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            
            for message in reversed(messages.data):
                if message.role == 'assistant':
                    response = message.content[0].text.value
                    cleaned_response = re.sub(r'[\x00-\x1f\x7f]', '', response)
                    ## genre, tone, viewpoint, story, summary는 string, keyword는 list형식으로 사용 가능
                    ## 프롬프팅처럼 Json Format형식으로 받게 설정 ex) response_story['keyword'] => ['a','b','c'] 형식으로 생성
                    response_json = json.loads(cleaned_response)
                    return response_json
                  
        else:
            return f"Error: Run failed with status {run.status}"
    except Exception as e:
        return f"Error: {str(e)}"


if __name__ == "__main__":
    user_input = input("프롬프트를 입력하세요: ")
    response_story = gpt_response_story(user_input)
    print(type(response_story))
    print("\nGPT의 답변:")
    print(json.dumps(response_story, indent=4, ensure_ascii=False))


<class 'dict'>

GPT의 답변:
{
    "genre": "공포",
    "tone": "반말",
    "viewpoint": "1인칭",
    "keywords": [
        "어둡고 깊은 구멍",
        "폐허 같은 집",
        "희미한 불빛",
        "차가운 바람",
        "낡은 나무 계단",
        "불길한 그림자"
    ],
    "story": "깊고 어두운 구멍 속으로 빨려들어가는 꿈을 꿨어. 내 몸이 점점 공중으로 떠오르더니, 갑자기 아래로 떨어지기 시작했지. 폐허가 된 집 안으로 추락하면서, 낡은 나무 계단이 부서지는 소리가 들려왔어. 희미한 불빛이 집 안을 비추었는데, 불빛 너머로 불길한 그림자가 서성거리고 있었어. 그 그림자가 내게 다가오며 차가운 바람이 불어오기 시작했어. 그림자는 점점 가까워지고, 난 도망치려 했지만 나의 발은 마치 땅에 묶인 듯 움직이지 않았어. 공포에 질린 나는 그저 그 그림자를 바라볼 수밖에 없었지. 마침내, 깨어나는 순간까지도 그 그림자는 내 눈앞에서 사라지지 않았어.",
    "summary": "In a dream, the narrator falls into a deep and dark hole, ending up in a dilapidated house. As they navigate the house, eerie shadows and cold wind heighten the sense of dread, with an ominous figure approaching, creating a tense and eerie atmosphere."
}


<h3>해몽 생섬 함수

In [ ]:
## 기본적인 틀은 스토리 생성 함수와 거의 동일 / 차이점 -> RAG사용, model변화, prompting 변화
## input 값이 text만 있어 매개변수는 하나로 설정
## token수 제약 때문인지 gpt-4o는 token 할당이 적어 RAG를 사용하지 못해 mini를 사용함
def gpt_response_interpret(user_input): 
    try:
        assistant = client.beta.assistants.create(            
            model="team2-gpt-mini",  
            instructions="""
            ## Role
            
            You are an expert in dream interpretation who provides symbolic meanings and concise interpretations of dreams based on the user's input. 
            Respond in the following JSON format.
            
            ## JSON Format
            
            {
            "interpretation": ""
            }
            
            ---
            
            ## Instructions
            
            1. The **interpretation** must always be written in **Korean**.
            2. Use polite and formal language (**존댓말**) for the interpretation.
            3. The interpretation should be concise and limited to approximately **2 sentences**.
            4. Use RAG (retrieval-augmented generation) to find symbolic meanings of dreams and incorporate them into the interpretation naturally.
            5. If no relevant data is found via RAG, create an original interpretation based on the user's dream input while strictly adhering to all restrictions.
            
            ---
            
            ## Restrictions
            
            1. Avoid mentioning "dream" as a theme or referring to waking up from a dream.
            
            ---
            
            ## Anonymity Guidelines
            
            1. Do not mention real-world locations (cities, countries, specific place names) or famous people (real individuals or well-known characters).
            2. Replace user-provided names or locations with fictional names or abstract descriptions.
            
            ---
            
            ### Important
            
            Adhere strictly to all instructions and restrictions outlined above.""",
            tools=[{"type":"file_search","file_search":{"ranking_options":{"ranker":"default_2024_08_21","score_threshold":0}}}],
            tool_resources={"file_search":{"vector_store_ids":["vs_pbD6MOjCO9mhML4X474CoAHQ"]}},
            temperature=1,
            top_p=1
          
        )
        
        
        
        thread = client.beta.threads.create()
        
        
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id
        )
        
    
        while run.status in ['queued', 'in_progress', 'cancelling']:
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
        
      
        if run.status == 'completed':
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            
            for message in reversed(messages.data):
                if message.role == 'assistant':
                    response = message.content[0].text.value
                    
                    ## 뒤에 참고자료가 나와서 정규표현식 사용
                    cleaned_response = re.sub(r'【.*?†source】', '', response)
                    response_json = json.loads(cleaned_response)
                    return response_json
                  
        else:
            return f"Error: Run failed with status {run.status}"
    except Exception as e:
        return f"Error: {str(e)}"
    
    
if __name__ == "__main__":
    user_input = input("호랑이와 고양이이: ")
    response_interpret = gpt_response_interpret(user_input)
    print(type(response_interpret))
    print("\nGPT의 답변:")
    print(json.dumps(response_interpret, indent=4, ensure_ascii=False))

In [34]:
type(response_interpret['interpretation'])

str